#### Made By:- Abhishek Sinha

As task for assessement for internship eligibility at AICT Pvt Ltd.

## Introduction
TASK 1: Movie Recommender System

Title: Build a Simple Movie Recommendation System<br>
Objective: Design a basic movie recommender using Python.

Instructions:
Use a public dataset like Movie Lens.

Build one of the following:<br>
 -> A content-based recommender (based on genres, keywords, etc.)<br>
 -> A collaborative filtering model (based on user ratings)<br>
Display top 5 recommended movies for a given input movie or user.
Bonus: Create a simple Stream lit or Flask UI.
Submission format:

Python/Jupyter Notebook (.ipynb or .py)
Output screenshots or short video (optional)
GitHub link (preferred) or Google Drive folder

Movie Recommender System

A movie recommender system is one which uses provided data to recommend movies to its user. Simple as that.<br>

Here the provided data can be data of the user's movie watching history or the data of movie reviews.<br>

I will be making a **content-based(item-based) movie recommender**

##### I used kaggle to run this first due to erformance constraints on my laptop

In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movielens-20m-dataset/rating.csv
/kaggle/input/movielens-20m-dataset/link.csv
/kaggle/input/movielens-20m-dataset/genome_tags.csv
/kaggle/input/movielens-20m-dataset/genome_scores.csv
/kaggle/input/movielens-20m-dataset/tag.csv
/kaggle/input/movielens-20m-dataset/movie.csv


In [3]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')

In [4]:
#load the data files
#genome_scores = pd.read_csv('/kaggle/input/movielens-20m-dataset/genome_scores.csv')
#genome_tags = pd.read_csv('/kaggle/input/movielens-20m-dataset/genome_tags.csv')
#link = pd.read_csv('/kaggle/input/movielens-20m-dataset/link.csv')
movie = pd.read_csv('/kaggle/input/movielens-20m-dataset/movie.csv')
#rating = pd.read_csv('/kaggle/input/movielens-20m-dataset/rating.csv')
#tag = pd.read_csv('/kaggle/input/movielens-20m-dataset/tag.csv')

In [5]:
movie.info()
movie.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  27278 non-null  int64 
 1   title    27278 non-null  object
 2   genres   27278 non-null  object
dtypes: int64(1), object(2)
memory usage: 639.5+ KB


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [6]:
#the datasets
data = movie
data.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [7]:
del movie

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  27278 non-null  int64 
 1   title    27278 non-null  object
 2   genres   27278 non-null  object
dtypes: int64(1), object(2)
memory usage: 639.5+ KB


### Right now I will use the genres of the movies to recommend the movies to user


In [9]:

#first encode the genres 
encoder = MultiLabelBinarizer()

data['genres'] = data['genres'].str.split('|')

encoded_genre = encoder.fit_transform(data['genres'])
encoded_genre

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [10]:
# here i am making the encoded_genre back into the DataFrame format
df_genre = pd.DataFrame(encoded_genre, columns = encoder.classes_, index = data.index)

#Joining it back to main dataset
data = pd.concat([data, df_genre], axis = 1)
data = data.drop(['genres'], axis = 1)

In [11]:
data.head()

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
#using corrwith isnt viable here so, I am using cosine similarity from sklearn

#first, get the genres values
genre_vals = df_genre.values

#then I can make the similarity matrix
similarity_matrix = cosine_similarity(genre_vals)

### Now the recommendation generator function based on the genres

#### Here the user input will be a movie that they have already watched.

In [13]:
def get_recommendations(movie_watched):
    #we get the indexes of that movie
    movie_index = data[data['title'] == MOVIE_NAME].index[0]
    
    #fetch similarity scores of that movie with others
    similarity_score = list(enumerate(similarity_matrix[movie_index]))

    #We do sorting of the movies in descending order of their similaity scores
    sorted_movies = sorted(similarity_score, key = lambda x: x[1], reverse = True)

    #Now we can get top 5 movies based on the most similarity scores
    #Of course we will exclude our input movie itself
    n = 5
    most_similar_movies = [data.iloc[i[0]]['title'] for i in sorted_movies[1:n+1]]

    print(f"Movies most similar to : '{movie_watched}'")
    for movie in most_similar_movies:
        print("--> ", movie)

# def getmovie_name(x):
#     return x[1]

In [14]:
#PLease enter the movie you have watched 
MOVIE_NAME = 'Jumanji (1995)'

get_recommendations(MOVIE_NAME)

Movies most similar to : 'Jumanji (1995)'
-->  Kids of the Round Table (1995)
-->  Indian in the Cupboard, The (1995)
-->  NeverEnding Story III, The (1994)
-->  Escape to Witch Mountain (1975)
-->  Darby O'Gill and the Little People (1959)


### CONCLUSION

There we have our required movies.
 
THANK YOU to anyone who has read this far into my notebook.
Please do feel free to comment your views into this as you want.